## ML Baseline: урезанный фичсет без групповой CV (без фильтра по АВТ)

Цель: получить сравнимую базовую линию на всём корпусе без текстового фильтра,
используя минимально достаточный набор признаков:
- геометрия: `cross_sectional_area`;
- материал: `material_resistance_score`;
- операционные: `operating_temperature`, `operating_pressure`;
- химические (2–3 ключевых): `sulfur_content`, `total_sulfur_compounds`, `h2s_content` (если доступны).

Здесь НЕ используем групповую валидацию по установкам — намеренно оцениваем упрощенный сценарий.


In [33]:
# Импорты
import os, sys, json
import numpy as np
import pandas as pd
from typing import List

# Визуализации опционально
import matplotlib.pyplot as plt
import seaborn as sns

# Данные
if '../src' not in sys.path:
    sys.path.append('../src')
from database import load_corrosion_data

# ML
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print('Версии:', {'python': sys.version, 'numpy': np.__version__, 'pandas': pd.__version__, 'sklearn': sklearn.__version__})


Версии: {'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'numpy': '2.3.4', 'pandas': '2.3.3', 'sklearn': '1.7.2'}


### Про нули в химических признаках

- Ноль трактуем как «компонент отсутствует» — это нормальная и полезная информация для модели.
- Пропуски (NaN) — это «значение не измерено»; мы имитируем отсутствие компонент значением 0, чтобы модель могла их использовать. Для линейных моделей дополнительно масштабируем признаки.
- Подробнее про импутацию: https://scikit-learn.org/stable/modules/impute.html


In [40]:
# Загрузка данных
df = load_corrosion_data()
print(f"Всего записей: {len(df):,}")

Всего записей: 442,052


In [41]:
# Работаем на всём корпусе без строкового фильтра (Ранее стаял фильтр по АВТ)
df_avt = df.copy()

TARGET = 'corrosion_rate'
df_avt = df_avt[df_avt[TARGET].notna() & (df_avt[TARGET] >= 0) & (df_avt[TARGET] <= 10)]
print(f"После фильтрации цели: {len(df_avt):,}")


После фильтрации цели: 313,132


In [42]:
# Формируем урезанный список признаков

base_features = [
    'cross_sectional_area',
    'material_resistance_score',
    'operating_temperature',
    'operating_pressure'
]

chemical_candidates = ['sulfur_content','total_sulfur_compounds','h2s_content','water_content','oxygen_content']

features = [c for c in base_features + chemical_candidates if c in df_avt.columns]
print('Признаки к использованию:', features)

X = df_avt[features].copy()
y = df_avt[TARGET].astype(float)

# Импутация: химические NaN -> 0 (нет данных ~ нет компонента), прочие NaN -> медиана
for c in features:
    if c in chemical_candidates:
        X[c] = X[c].fillna(0)
    else:
        X[c] = X[c].fillna(X[c].median())

print('Проверка пропусков после импутации:', int(X.isna().sum().sum()))


Признаки к использованию: ['cross_sectional_area', 'material_resistance_score', 'operating_temperature', 'operating_pressure', 'sulfur_content', 'total_sulfur_compounds', 'h2s_content', 'water_content', 'oxygen_content']
Проверка пропусков после импутации: 0


In [43]:
# Взаимодействия химия × температура

interaction_features = []
if 'operating_temperature' in X.columns:
    if 'water_content' in X.columns:
        name = 'water_content_x_operating_temperature'
        X[name] = X['water_content'] * X['operating_temperature']
        interaction_features.append(name)
    if 'oxygen_content' in X.columns:
        name = 'oxygen_content_x_operating_temperature'
        X[name] = X['oxygen_content'] * X['operating_temperature']
        interaction_features.append(name)

print('Добавлены взаимодействия:', interaction_features)


Добавлены взаимодействия: ['water_content_x_operating_temperature', 'oxygen_content_x_operating_temperature']


In [44]:
# Модели по каждой installation: отдельные метрики

from sklearn.model_selection import train_test_split

key_col = 'installation' if 'installation' in df_avt.columns else ('equipment' if 'equipment' in df_avt.columns else None)
if key_col is None:
    raise ValueError('Нет поля installation/equipment для группировки')

per_results = []
min_samples = 150  # минимальный размер выборки для обучения на установке

unique_keys = (
    df_avt[key_col].dropna().astype(str).unique()
)

for k in sorted(unique_keys):
    mask_k = df_avt[key_col].astype(str) == k
    n_k = int(mask_k.sum())
    if n_k < min_samples:
        continue

    Xk = X.loc[mask_k]
    yk = y.loc[mask_k]

    X_tr, X_va, y_tr, y_va = train_test_split(Xk, yk, test_size=0.2, random_state=42)

    # Ridge
    ridge = Pipeline([('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
    ridge.fit(X_tr, y_tr)
    pr = ridge.predict(X_va)
    ridge_mae = mean_absolute_error(y_va, pr)
    ridge_rmse = np.sqrt(mean_squared_error(y_va, pr))
    ridge_r2 = r2_score(y_va, pr)

    # RandomForest
    rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
    rf.fit(X_tr, y_tr)
    prf = rf.predict(X_va)
    rf_mae = mean_absolute_error(y_va, prf)
    rf_rmse = np.sqrt(mean_squared_error(y_va, prf))
    rf_r2 = r2_score(y_va, prf)

    per_results.append({
        key_col: k,
        'n_samples': n_k,
        'Ridge_MAE': float(ridge_mae),
        'Ridge_RMSE': float(ridge_rmse),
        'Ridge_R2': float(ridge_r2),
        'RF_MAE': float(rf_mae),
        'RF_RMSE': float(rf_rmse),
        'RF_R2': float(rf_r2)
    })

per_df = pd.DataFrame(per_results).sort_values(['RF_R2','Ridge_R2'], ascending=False)
print('ТОП-10 установок по RF_R2:')
print(per_df.head(10).to_string(index=False))

# Сохранение
os.makedirs('../data', exist_ok=True)
per_json = '../data/ml_baseline_avt_per_installation.json'
per_csv = '../data/ml_baseline_avt_per_installation.csv'
with open(per_json, 'w', encoding='utf-8') as f:
    json.dump(per_results, f, ensure_ascii=False, indent=2)
per_df.to_csv(per_csv, index=False, encoding='utf-8')
print(f'Сохранено: {per_json}\nСохранено: {per_csv}')


ТОП-10 установок по RF_R2:
installation  n_samples  Ridge_MAE  Ridge_RMSE  Ridge_R2   RF_MAE  RF_RMSE    RF_R2
        KK-2      37814   0.047469    0.077337  0.019540 0.043535 0.064476 0.318520
       АВТ-5      59667   0.046341    0.090325  0.036232 0.041681 0.080363 0.237095
          КК      97477   0.038035    0.069981  0.007891 0.037224 0.066590 0.101717
       АВТ-6      74299   0.066174    0.162034  0.021974 0.061752 0.157295 0.078349
       АВТ-2      24963   0.220496    0.471466  0.009017 0.212879 0.458253 0.063787
       АВТ-1      18912   0.068123    0.234358  0.022447 0.065542 0.231256 0.048149
Сохранено: ../data/ml_baseline_avt_per_installation.json
Сохранено: ../data/ml_baseline_avt_per_installation.csv
